# Assignment 4


This module we'll be looking at the New York City tree census. This data was provided by a volunteer driven census in 2015, and we'll be accessing it via the socrata API. The main site for the data is here, and on the upper right hand side you'll be able to see the link to the API.

In [1]:
import pandas as pd
import numpy as np

In [2]:
url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json'
trees = pd.read_json(url)
trees.head(10)
trees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 45 columns):
address             1000 non-null object
bbl                 987 non-null float64
bin                 987 non-null float64
block_id            1000 non-null int64
boro_ct             1000 non-null int64
borocode            1000 non-null int64
boroname            1000 non-null object
brch_light          1000 non-null object
brch_other          1000 non-null object
brch_shoe           1000 non-null object
cb_num              1000 non-null int64
census_tract        989 non-null float64
cncldist            1000 non-null int64
council_district    989 non-null float64
created_at          1000 non-null datetime64[ns]
curb_loc            1000 non-null object
guards              973 non-null object
health              973 non-null object
latitude            1000 non-null float64
longitude           1000 non-null float64
nta                 1000 non-null object
nta_name            1000 non-n

In [3]:
trees.shape

(1000, 45)


1000 seems like too few trees for a city like New York, and a suspiciously round number. What's going on?

Socrata places a 1000 row limit on their API. Raw data is meant to be "paged" through for applications, with the expectation that a UX wouldn't be able to handle a full dataset.

As a simple example, if we had a mobile app with limited space that only displayed trees 5 at a time, we could view the first 5 trees in the dataset with the url below:

In [4]:
firstfive_url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=0'
firstfive_trees = pd.read_json(firstfive_url)
firstfive_trees

,address,bbl,bin,block_id,boro_ct,borocode,boroname,brch_light,brch_other,brch_shoe,...,tree_dbh,tree_id,trnk_light,trnk_other,trunk_wire,user_type,x_sp,y_sp,zip_city,zipcode
0,108-005 70 AVENUE,4022210001,4052307,348711,4073900,4,Queens,No,No,No,...,3,180683,No,No,No,TreesCount Staff,1027431.148,202756.7687,Forest Hills,11375
1,147-074 7 AVENUE,4044750045,4101931,315986,4097300,4,Queens,No,No,No,...,21,200540,No,No,No,TreesCount Staff,1034455.701,228644.8374,Whitestone,11357
2,390 MORGAN AVENUE,3028870001,3338310,218365,3044900,3,Brooklyn,No,No,No,...,3,204026,No,No,No,Volunteer,1001822.831,200716.8913,Brooklyn,11211
3,1027 GRAND STREET,3029250001,3338342,217969,3044900,3,Brooklyn,No,No,No,...,10,204337,No,No,No,Volunteer,1002420.358,199244.2531,Brooklyn,11211
4,603 6 STREET,3010850052,3025654,223043,3016500,3,Brooklyn,No,No,No,...,21,189565,No,No,No,Volunteer,990913.775,182202.4260,Brooklyn,11215


If we wanted the next 5, we would use this url:

In [5]:
nextfive_url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=5'
nextfive_trees = pd.read_json(nextfive_url)
nextfive_trees

,address,bbl,bin,block_id,boro_ct,borocode,boroname,brch_light,brch_other,brch_shoe,...,tree_dbh,tree_id,trnk_light,trnk_other,trunk_wire,user_type,x_sp,y_sp,zip_city,zipcode
0,8 COLUMBUS AVENUE,1.011310e+09,1076229.0,106099,1014500,1,Manhattan,No,No,No,...,11,190422,No,No,No,Volunteer,988418.6997,219825.5227,New York,10023
1,120 WEST 60 STREET,1.011310e+09,1076229.0,106099,1014500,1,Manhattan,No,No,No,...,11,190426,No,No,No,Volunteer,988311.1900,219885.2785,New York,10023
2,311 WEST 50 STREET,1.010410e+09,1086093.0,103940,1012700,1,Manhattan,No,No,No,...,9,208649,No,No,No,Volunteer,987769.1163,217157.8561,New York,10019
3,65 JEROME AVENUE,NaN,NaN,407443,5006400,5,Staten Island,No,No,No,...,6,209610,No,No,No,TreesCount Staff,963073.1998,156635.5542,Staten Island,10305
4,638 AVENUE Z,3.072350e+09,3320727.0,207508,3037402,3,Brooklyn,No,No,No,...,21,192755,No,No,No,TreesCount Staff,992653.7253,152903.6306,Brooklyn,11223



You can read more about paging using the Socrata API here

In these docs, you'll also see more advanced functions (called SoQL) under the "filtering and query" section. These functions should be reminding you of SQL.

Think about the shape you want your data to be in before querying it. Using SoQL is a good way to avoid the limits of the API. For example, using the below query I can easily obtain the count of each species of tree in the Bronx:

In [6]:
boro = 'Bronx'
soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,count(tree_id)' +\
        '&$where=boroname=\'Bronx\'' +\
        '&$group=spc_common').replace(' ', '%20')
soql_trees = pd.read_json(soql_url)

soql_trees

,count_tree_id,spc_common
0,4619,NaN
1,662,silver maple
2,18,pagoda dogwood
3,3917,littleleaf linden
4,12,American larch
5,1483,northern red oak
6,1889,green ash
7,7,pignut hickory
8,56,eastern cottonwood
9,177,shingle oak


This behavior is very common with web APIs, and I think this is useful when thinking about building interactive data products. When in a Jupyter Notebook or RStudio, there's an expectation that (unless you're dealing with truly large datasets) the data you want can be brought in memory and manipulated.

Dash and Shiny abstract away the need to distinguish between client side and server side to make web development more accessible to data scientists. This can lead to some unintentional design mistakes if you don't think about how costly your callback functions are (for example: nothing will stop you in dash from running a costly model triggered whenever a dropdown is called.)

The goal of using the Socrata is to force you to think about where your data operations are happening, and not resort to pulling in the data and performing all operations in local memory.

NOTE: One tip in dealing with URLs: you may need to replace spaces with '%20'. I personally just write out the url and then follow the string with a replace:

In [7]:

'https://api-url.com/?query with spaces'.replace(' ', '%20')

'https://api-url.com/?query%20with%20spaces'

In [2]:
#loading the needed packages 
from jupyter_plotly_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import multiprocessing


In [3]:
soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
            '$select=health,count(tree_id)' +\
            '&$group=health').replace(' ', '%20')
soql_trees = pd.read_json(soql_url)

soql_trees

,count_tree_id,health
0,96504,Fair
1,528850,Good
2,26818,Poor
3,31616,NaN


In [10]:
from IPython import display
import os
def show_app(app, port = 9999, 
             width = 700, 
             height = 350, 
             offline = False,
            in_binder = None):
    in_binder ='JUPYTERHUB_SERVICE_PREFIX' in os.environ if in_binder is None else in_binder
    if in_binder:
        base_prefix = '{}proxy/{}/'.format(os.environ['JUPYTERHUB_SERVICE_PREFIX'], port)
        url = 'https://hub.mybinder.org{}'.format(base_prefix)
        app.config.requests_pathname_prefix = base_prefix
    else:
        url = 'http://localhost:%d' % port
        
    iframe = '<a href="{url}" target="_new">Open in new window</a><hr><iframe src="{url}" width={width} height={height}></iframe>'.format(url = url, 
                                                                                  width = width, 
                                                                                  height = height)
    
    display.display_html(iframe, raw = True)
    if offline:
        app.css.config.serve_locally = True
        app.scripts.config.serve_locally = True
    return app.run_server(debug=False, # needs to be false in Jupyter
                          host = '0.0.0.0',
                          port=port)

In [ ]:
app_simple = dash.Dash()

app_simple.layout = html.Div(children=[
    html.H1(children='Health Condition'),

    html.Div(children='''
        Proportion of Trees 
    '''),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [

            ],
            'layout': {
                'title': 'Tree Data Visualization'
            }
        }
    )
])

In [5]:
health = 'Fair'
soql_url1 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=steward,count(tree_id)' +\
        '&$where=health=\'Fair\'' +\
        '&$group=steward').replace(' ', '%20')
soql_trees1 = pd.read_json(soql_url1)

soql_trees1

,count_tree_id,steward
0,2830,3or4
1,200,4orMore
2,21902,1or2
3,71572,None


In [6]:
health = 'Good'
soql_url2 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=steward,count(tree_id)' +\
        '&$where=health=\'Good\'' +\
        '&$group=steward').replace(' ', '%20')
soql_trees2 = pd.read_json(soql_url2)

soql_trees2

,count_tree_id,steward
0,115372,1or2
1,15606,3or4
2,1361,4orMore
3,396511,None


In [7]:
health = 'Poor'
soql_url3 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=steward,count(tree_id)' +\
        '&$where=health=\'Poor\'' +\
        '&$group=steward').replace(' ', '%20')
soql_trees3 = pd.read_json(soql_url3)

soql_trees3

,count_tree_id,steward
0,747,3or4
1,49,4orMore
2,6283,1or2
3,19739,None
